In [9]:

#pd.set_option('display.max_colwidth', None)

import time
import scipy
import string 
import pickle
import random
import argparse
import itertools
import numpy as np
import pandas as pd

from pprint import pprint

from tqdm.notebook import tqdm

from scipy.stats import spearmanr

from gensim.models import KeyedVectors

from sentence_transformers import models, SentenceTransformer

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor, LogisticRegression, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

# from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import RegexpTokenizer


In [10]:
df_scores=pd.read_csv('data_2021/TRECVid Data/training_set/train_scores.csv')
#df_scores=pd.read_csv('data_2021/Memento10k Data/training_set/train_scores.csv')
df_scores

,video_id,video_url,annotations_short_term,annotations_long_term,scores_raw_short_term,scores_raw_long_term,scores_normalized_short_term,decay_alpha
0,8,https://mtc.cdn.vine.co/r/videos_h264high/9EB0...,20,7,0.75,0.57,0.624240,-1.289843e-02
1,26,https://mtc.cdn.vine.co/r/videos_h264high/A8B3...,23,7,0.87,0.43,0.869565,0.000000e+00
2,33,https://mtc.cdn.vine.co/r/videos/267829AEFA128...,26,4,0.69,0.75,0.672876,-1.279030e-03
3,46,https://mtc.cdn.vine.co/r/videos_h264high/B974...,23,7,0.87,0.57,0.869565,0.000000e+00
4,64,https://mtc.cdn.vine.co/r/videos_h264high/C4D6...,19,9,0.84,0.56,0.769838,-1.163618e-02
...,...,...,...,...,...,...,...,...
583,6136,https://mtc.cdn.vine.co/r/videos_h264high/2D38...,20,7,0.90,1.00,0.900000,0.000000e+00
584,6154,https://v.cdn.vine.co/r/videos/09E48CC7D810160...,21,9,0.90,1.00,0.904762,0.000000e+00
585,6177,https://mtc.cdn.vine.co/r/videos_h264high/12B3...,23,6,0.96,0.83,0.956522,0.000000e+00
586,6182,https://mtc.cdn.vine.co/r/videos/AA485CE97B136...,17,6,1.00,0.67,1.000000,-1.431271e-08


In [11]:
df_text=pd.read_csv('data_2021/TRECVid Data/training_set/train_text_descriptions.csv')



#df_text=pd.read_csv('data_2021/Memento10k Data/training_set/train_text_descriptions.csv')
#df_text['description']=df_text['description_0']+df_text['description_1']+df_text['description_2']+df_text['description_3']+df_text['description_4']

#train_text_descriptions=df.groupby('id').agg(lambda x: x.tolist())


In [12]:
test_text_descriptions=pd.read_csv('data_2021/TRECVid Data/test_set/test_text_descriptions.csv')


In [13]:
lemmatizer = WordNetLemmatizer()
spearman = lambda x,y: spearmanr(x, y).correlation
stopwords = ["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would"]

def tokenize(s):
    numbers = {'2': 'two', '3': 'three', '4': 'four'}
    s = ''.join(c for c in s if c not in string.punctuation or c == ' ').lower()
    t = RegexpTokenizer(r'\w+').tokenize(s)
    t = [lemmatizer.lemmatize(w) if w not in numbers else numbers[w] for w in t if w not in stopwords]
    return ' '.join(t)

def tokenize2(s): # keep stopwords and don't lemmatize
    numbers = {'2': 'two', '3': 'three', '4': 'four'}
    s = ''.join(c for c in s if c not in string.punctuation or c == ' ').lower()
    t = RegexpTokenizer(r'\w+').tokenize(s)
    t = [w if w not in numbers else numbers[w] for w in t]
    return ' '.join(t)

def tokenize3(s): # remove duplicates
    numbers = {'2': 'two', '3': 'three', '4': 'four'}
    s = ''.join(c for c in s if c not in string.punctuation or c == ' ').lower()
    t = RegexpTokenizer(r'\w+').tokenize(s)
    t = [w if w not in numbers else numbers[w] for w in t if w not in stopwords]
    return ' '.join(set(t))






In [14]:



# For videos having multiple descriptions
text_concat = df_text[['video_id','description']].groupby(['video_id'])['description'].transform(lambda x: ' '.join(x)).drop_duplicates()

#deep_captions = [ l[8:].strip() for l in open(args.deep_caption_path)]

df_data = df_scores.copy()
df_data['text'] = text_concat.values
df_data['short_term'] = df_data['scores_raw_short_term']
if 'scores_raw_long_term' in df_data.columns:
    df_data['long_term'] = df_data['scores_raw_long_term']
#df_data['deep_caption'] = deep_captions[:590]
#df_data = df_data[['video_id', 'text', 'deep_caption', 'short_term', 'long_term']]
if 'long_term' in df_data.columns:
    df_data = df_data[['video_id', 'text', 'short_term', 'long_term']]
else:
    df_data = df_data[['video_id', 'text', 'short_term']]
    
df_data['content'] = df_data['text'] #+ '  ' + df_data['deep_caption']

In [15]:
corpus = df_data.text.values
corpus_tokenized = [tokenize3(s) for s in corpus]
#corpus_tokenized2 = [tokenize2(s) for s in corpus]
#corpus_tokenized3 = [tokenize3(s) for s in corpus]

vectorizer = TfidfVectorizer(min_df=4, stop_words='english', ngram_range=(1, 2))
cv=vectorizer.fit(corpus_tokenized)
print(list(cv.vocabulary_.keys())[:30])

X_train = vectorizer.transform(corpus)

print(X_train.shape)
print(X_train.toarray())
#w2v = pickle.load(open('/data/glove.6B/glove.w2v.6B.300d.pickle', 'rb'))


X_tfidf = []
X_w2v   = []
Y = []


for i, entry in df_data.iterrows():
    text = tokenize(entry['text'])
    if 'long_term' in df_data.columns:
        y = (entry['short_term'], entry['long_term'])
    else:
        y = (entry['short_term'],'no_lt')
        
    
    x_tfidf = vectorizer.transform([text]).toarray()[0]
    #words = [word for word in text.split(' ') if word in w2v]
    #x_w2v = np.zeros([300]) if not words else np.mean([w2v[word] for word in words], axis=0)

    
    X_tfidf.append(x_tfidf)
    #X_w2v.append(x_w2v)
    Y.append(y)

X_tfidf = np.array(X_tfidf)
#X_w2v = np.array(X_w2v)
Y = np.array(Y)

X_tfidf.shape, Y.shape



['young', 'ass', 'hug', 'men', 'man', 'standing', 'claps', 'away', 'stage', 'walk', 'costumes', 'persons', 'dance', 'cup', 'rolling', 'soccer', 'field', 'daytime', 'player', 'goal', 'small', 'shoots', 'ball', 'field daytime', 'daytime player', 'player goal', 'podium', 'onstage', 'perform', 'person']
(588, 930)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


((588, 930), (588, 2))

In [17]:
def enumerate_models(models):
    instances = []
    for model_name, (model, hyperparameters) in models.items():
        configs = {}
        if len(hyperparameters) > 0:
            params, vals = list(hyperparameters.keys()), list(hyperparameters.values())
            configs = [dict(zip(params, vv)) for vv in list(itertools.product(*vals))]
            for config in configs:
                m = model(**config)
                instances.append(m)
        else:
            instances.append(model())
    return instances


regression_models = {
    # 'LogisticRegression': (LogisticRegression, {"C": [1e3, 1, 1e-3], "penalty": ['l1', 'l2', 'elasticnet']}),
    # 'LinearRegression': (LinearRegression, {}),
    # 'MLPRegressor': (MLPRegressor, {'alpha': [1e-3,  1e-7], 'hidden_layer_sizes': [(10,), (100,)]}), # 1e-5,, (50,), 
    # 'SGDRegressor': (SGDRegressor, {'alpha': [0.0001, 0.1,]}),
    # 'SVR': (SVR, {'kernel': ['linear', 'rbf'], "C": [1e-3, 1e-4, 1e-5, 1e-7], "gamma": ["scale"]})
    'SVR': (SVR, {'kernel': ['linear',], "C": [1e-3, 1e-5, 1e-7], "gamma": ["scale"]})
}
len(enumerate_models(regression_models))


X = {'tfidf': X_tfidf}#, 'w2v':X_w2v, 'bert1': bert1_embeddings, 'bert2': bert2_embeddings}
Y_st = Y[:, 0]
Y_lt = Y[:, 1]


folds = {}
print('Short term memorability prediction:'.upper())

for k in X:
    folds[k] = {}
    print('\nFeatures:', k.upper(), '\n')
    for regressor in enumerate_models(regression_models):
        model_name = str(regressor)
        folds[k][model_name] = []
        kf = KFold(n_splits=6)#, random_state=42)
        print('Training', model_name, '..')
        for i, (train_index, test_index) in enumerate(kf.split(X[k])):
            print('Fold #'+ str(i), end='.. ')
            t = time.time()
            
            X_train, X_test = X[k][train_index], X[k][test_index]
            y_train, y_test = Y_st[train_index], Y_st[test_index]
            regressor.fit(X_train, y_train)
            important_tokens = pd.DataFrame(data=regressor.coef_[0],index=vectorizer.get_feature_names(),
            columns=['coefficient']).sort_values(by='coefficient',ascending=False)
            print(important_tokens[0:30])
            print(important_tokens[-30:])
            y_pred = regressor.predict(X_test)
            folds[k][model_name].append((y_pred, y_test))
            print(f'done! ({(time.time() - t):.2} secs). Spearman: {spearman(y_pred, y_test):.2}')
            
            t = time.time()


folds_lt = {}
print('Long term memorability prediction:'.upper())

for k in X:
    folds_lt[k] = {}
    print('\nFeatures:', k.upper(), '\n')
    for regressor in enumerate_models(regression_models):
        model_name = str(regressor)
        folds_lt[k][model_name] = []
        kf = KFold(n_splits=6)#, random_state=42)
        print('Training', model_name, '..')
        for i, (train_index, test_index) in enumerate(kf.split(X[k])):
            print('Fold #'+ str(i), end='.. ')
            t = time.time()
            X_train, X_test = X[k][train_index], X[k][test_index]
            y_train, y_test = Y_lt[train_index], Y_lt[test_index]
            regressor.fit(X_train, y_train)
            
            y_pred = regressor.predict(X_test)
            folds_lt[k][model_name].append((y_pred, y_test))
            print(f'done! ({(time.time() - t):.2} secs). Spearman: {spearman(y_pred, y_test):.2}')

            
            t = time.time()

            
for term, all_folds in [('Short term', folds), ('Long term', folds_lt), ('Long Short term', folds_lt)]:
    print(term.upper())
    for embedding in all_folds:
        print('  USING', embedding)
        for i, model_name in enumerate(all_folds[embedding]):
            # print('    ', (model_name.split('(')[0] + ' ' + str(i+1)).ljust(18), '\t', end=' ')
            print('    ', model_name, '\t', end=' ')
            # print(', '.join([str(spearman(y_p, y_t)) for y_p, y_t in all_folds[embedding][model_name]]))
            if term == 'Long Short term':
                sps = [spearman(ys_p, yl_t) for (ys_p, ys_t), (yl_p, yl_t)
                       in zip(folds[embedding][model_name], folds_lt[embedding][model_name])]
            else:
                sps = [spearman(y_p, y_t) for y_p, y_t in all_folds[embedding][model_name]]
            print(round(sum(sps)/len(sps), 4))


SHORT TERM MEMORABILITY PREDICTION:

Features: TFIDF 

Training SVR(C=0.001, kernel='linear') ..
Fold #0..            coefficient
tv            0.002028
slide         0.001768
finger        0.001617
car           0.001474
leg           0.001455
laugh         0.001407
baby          0.001164
phone         0.001158
talking       0.001110
blue          0.001098
child         0.001092
video         0.001062
throw         0.001032
screen        0.001020
skirt         0.001015
open          0.001014
door          0.000977
hold          0.000973
small         0.000929
sing          0.000922
listening     0.000858
driver        0.000858
singing       0.000853
wheel         0.000848
human         0.000840
green         0.000819
person        0.000812
head          0.000803
music         0.000798
floor         0.000797
          coefficient
parking     -0.000833
japanese    -0.000846
looking     -0.000856
couple      -0.000871
turn        -0.000875
black       -0.000896
beach       -0.000911
base

/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/si

                 coefficient
laugh               0.001997
leg                 0.001939
open                0.001795
tv                  0.001718
slide               0.001615
daytime             0.001488
turtle              0.001388
car                 0.001339
mouth               0.001337
person              0.001281
kitchen             0.001277
child               0.001205
baby                0.001164
throw               0.001158
finger              0.001153
video               0.001139
talking             0.001027
athlete             0.000900
studio              0.000876
green               0.000875
cloudy              0.000837
away                0.000827
hold                0.000819
toddler             0.000817
sings               0.000804
indoors daytime     0.000758
point               0.000745
phone               0.000729
plastic             0.000720
listening           0.000709
               coefficient
girl             -0.000768
male             -0.000825
ride             -0.

/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/si

           coefficient
open          0.000022
laugh         0.000021
leg           0.000017
finger        0.000016
daytime       0.000014
turtle        0.000014
slide         0.000014
person        0.000013
kitchen       0.000013
face          0.000013
blue          0.000012
throw         0.000012
door          0.000010
skirt         0.000010
tv            0.000010
hold          0.000010
car           0.000009
video         0.000009
mouth         0.000009
run           0.000009
listening     0.000009
human         0.000008
cloudy        0.000008
moving        0.000008
green         0.000008
baby          0.000008
small         0.000008
window        0.000008
point         0.000007
away          0.000007
               coefficient
young            -0.000008
skate            -0.000008
black            -0.000008
ball             -0.000008
air              -0.000008
white            -0.000008
audience         -0.000008
parking          -0.000008
shown            -0.000008
field daytime    

...

           coefficient
car           0.000025
open          0.000022
leg           0.000020
slide         0.000018
phone         0.000018
finger        0.000016
tv            0.000015
turtle        0.000014
person        0.000014
kitchen       0.000013
video         0.000012
stand         0.000012
daytime       0.000012
driver        0.000011
laugh         0.000011
moving        0.000010
boy           0.000009
ring          0.000009
girl          0.000009
door          0.000009
listening     0.000009
wheel         0.000008
skirt         0.000008
human         0.000008
mouth         0.000008
sing          0.000008
hold          0.000008
seat          0.000008
screen        0.000008
toddler       0.000008
               coefficient
outdoor          -0.000008
asian            -0.000008
team             -0.000009
group            -0.000009
crowd            -0.000009
cheering         -0.000009
air              -0.000009
field daytime    -0.000009
beach            -0.000009
people           

/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/semantic/.conda/envs/memorability21/lib/python3.9/si

                  coefficient
laugh            1.996953e-07
leg              1.939214e-07
open             1.795348e-07
slide            1.615290e-07
tv               1.525368e-07
daytime          1.487793e-07
turtle           1.387502e-07
car              1.338888e-07
mouth            1.337197e-07
kitchen          1.277341e-07
child            1.204510e-07
baby             1.164324e-07
throw            1.157728e-07
finger           1.152735e-07
video            1.139465e-07
person           1.124199e-07
talking          1.026916e-07
athlete          8.996412e-08
studio           8.760913e-08
green            8.753847e-08
cloudy           8.374900e-08
away             8.272164e-08
hold             8.185020e-08
toddler          8.171072e-08
sings            8.044361e-08
indoors daytime  7.575815e-08
point            7.445610e-08
phone            7.289245e-08
plastic          7.203717e-08
listening        7.089528e-08
                coefficient
male          -8.249204e-08
ride          

/home/semantic/.conda/envs/memorability21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


done! (0.077 secs). Spearman: -0.038
Fold #1.. done! (0.067 secs). Spearman: 0.0095
Fold #2.. done! (0.073 secs). Spearman: -0.078
Fold #3.. done! (0.075 secs). Spearman: 0.12
Fold #4.. done! (0.075 secs). Spearman: 0.0079
Fold #5.. done! (0.074 secs). Spearman: 0.015
Training SVR(C=1e-05, kernel='linear') ..
Fold #0.. done! (0.073 secs). Spearman: -0.043
Fold #1.. done! (0.07 secs). Spearman: 0.0095
Fold #2.. done! (0.075 secs). Spearman: -0.078
Fold #3.. done! (0.083 secs). Spearman: 0.11
Fold #4.. done! (0.091 secs). Spearman: 0.0079
Fold #5.. done! (0.088 secs). Spearman: 0.017
Training SVR(C=1e-07, kernel='linear') ..
Fold #0.. done! (0.087 secs). Spearman: -0.043
Fold #1.. done! (0.078 secs). Spearman: 0.0095
Fold #2.. done! (0.062 secs). Spearman: -0.078
Fold #3.. done! (0.084 secs). Spearman: 0.11
Fold #4.. done! (0.091 secs). Spearman: 0.0079
Fold #5.. done! (0.087 secs). Spearman: 0.017
SHORT TERM
  USING tfidf
     SVR(C=0.001, kernel='linear') 	 0.1772
     SVR(C=1e-05, ker

In [ ]:
woman           0.027050
man             0.021254
girl            0.009931
using           0.009736
black           0.009590
baby            0.009285
hand            0.008833
piece           0.008495
cat             0.008259
shirt           0.008093
machine         0.007570
dog             0.007490
red             0.007462
weight          0.007433
dress           0.006908
lady            0.006811
food            0.006685
smile           0.006132
hold            0.005869
hair            0.005764
push            0.005683
young           0.005341
brown           0.005287
metal           0.005124
toy             0.005054
lift            0.005040
man wearing     0.004771
white           0.004735
pink            0.004698
sitting         0.004674
done! (9.3e+01 secs). Spearman: 0.46
            coefficient
woman       0.026767
man         0.017613
black       0.010871
using       0.010502
piece       0.009759
cat         0.008430
girl        0.008265
hand        0.007965
dress       0.007930
lady        0.007490
weight      0.007433
baby        0.006793
smile       0.006459
wooden      0.006425
shirt       0.006305
dog         0.006287
machine     0.006039
hair        0.005767
lift        0.005672
toy         0.005340
push        0.005177
head        0.005082
blonde      0.005049
metal       0.004991
roll        0.004873
brown       0.004841
doll        0.004829
hold        0.004595
young       0.004549
box         0.004546
done! (9.6e+01 secs). Spearman: 0.54
    
    snow         -0.008271
pan          -0.008348
player       -0.008407
soccer       -0.008594
skiing       -0.008766
ocean        -0.008870
large        -0.008934
band         -0.008980
football     -0.009054
crowd        -0.009117
driving      -0.009170
ride         -0.009203
stage        -0.009378
road         -0.009425
race         -0.009509
street       -0.009620
waterfall    -0.009799
rock         -0.009804
dark         -0.009942
field        -0.010205
wave         -0.011475
hill         -0.011821
snowy        -0.012301
ski          -0.012648
view         -0.013653
car          -0.016878
tree         -0.016965
mountain     -0.019070
group        -0.022045
people       -0.038964